In [1]:
pip install geoclip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 35.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -q geograpy3 gensim nltk ndg-httpsclient pyopenssl pyasn1

Note: you may need to restart the kernel to use updated packages.


In [4]:
from tqdm import tqdm

In [5]:
import os
import csv
import torch
from geoclip import GeoCLIP
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

model = GeoCLIP().to("cuda")
geolocator = Nominatim(user_agent="GEOCLIPOSINT")

folder_path = "/kaggle/input/llms-for-image-geolocation-benchmarks/yfcc4k/yfcc4k/yfcc4k/"
output_csv = "/kaggle/working/geoclip_predictions_yfcc.csv"

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

2024-08-03 09:35:30.158697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 09:35:30.158837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 09:35:30.283732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:

# Function to get address from coordinates
def get_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en', zoom=8)
        return location.address if location else "Address not found"
    except:
        try:
            location = geolocator.reverse((lat, lon), language='en', zoom=8)
        except:   
            location = False
        return location.address if location else "Address not found"

# Open a CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    header = ["Image"]
    for i in range(1, 6):
        header.extend([f"Prediction {i} Latitude", f"Prediction {i} Longitude", f"Prediction {i} Probability", f"Prediction {i} Address"])
    writer.writerow(header)
    
    # Iterate through each image
    for image_file in tqdm(image_files):
        image_path = os.path.join(folder_path, image_file)
        
        top_pred_gps, top_pred_prob = model.predict(image_path, top_k=5)
        
        # Prepare a row for the current image
        row = [image_file]
        for i in range(5):
            lat, lon = top_pred_gps[i]
            lat = str(float(lat))
            lon = str(float(lon))
            prob = top_pred_prob[i]
            address = get_address(lat, lon)
            print(address)
            row.extend([lat, lon, prob, address])
        
        # Write the row to the CSV file
        writer.writerow(row)

print(f"Predictions saved to {output_csv}")


  0%|          | 0/4536 [00:00<?, ?it/s]

North Jakarta, Special Region of Jakarta, Java, Indonesia
Bandung, West Java, Java, Indonesia
Bandung, West Java, Java, Indonesia
East Jakarta, Special Region of Jakarta, Java, Indonesia


  0%|          | 1/4536 [00:03<4:38:19,  3.68s/it]

Surabaya, East Java, Java, Indonesia
Davao City, Davao Region, Philippines
Davao City, Davao Region, Philippines
Partido de Marcos Paz, Buenos Aires, Argentina
Partido de Marcos Paz, Buenos Aires, Argentina


  0%|          | 2/4536 [00:06<3:47:48,  3.01s/it]

...

Ottawa County, Michigan, United States
England, United Kingdom
England, United Kingdom
Metropolitan France, France
Metropolitan France, France


 71%|███████   | 3217/4536 [2:14:39<55:02,  2.50s/it]

Metropolitan France, France
Unorganized Borough, Alaska, United States
Unorganized Borough, Alaska, United States
Ketchikan Gateway, Alaska, United States
Unorganized Borough, Alaska, United States


 71%|███████   | 3218/4536 [2:14:41<54:49,  2.50s/it]

Ketchikan Gateway, Alaska, United States
Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Federal District, Central-West Region, Brazil
Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Federal District, Central-West Region, Brazil
Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Federal District, Central-West Region, Brazil
